# Journey 5: Orchestrate with MLOps
This journey is about automating Journey 1-4 with MLOps and corresponds to JTBD of MLOps orchestration in Pillar 2. Thus, we automate multi-model evaluation, fine-tuning, selection, versioning/registration, and deployment.
This notebook is using different Amazon SageMaker components:

### SageMaker Pipelines

[SageMaker Pipelines](https://aws.amazon.com/sagemaker/pipelines/) is a purpose-built workflow orchestration service to automate all phases of machine learning (ML) from data pre-processing to model monitoring. With an intuitive UI and Python SDK you can manage repeatable end-to-end ML pipelines at scale. The native integration with multiple AWS services allows you to customize the ML lifecycle based on your MLOps requirements.
SageMaker Model Registry

### Amazon SageMaker Model Registry 

[Amazon SageMaker Model Registry](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry.html) is a purpose-built metadata store to manage the entire lifecycle of ML models from training to inference. Whether you prefer to store your model artifacts (model framework files, container image) in AWS (Amazon ECR) or outside of AWS in any third party Docker repository, you can now track them all in Amazon SageMaker Model Registry. You also have the flexibility to register a model without read/write permissions to the associated container image. If you want to track an ML model in a private repository, set the optional ‘SkipModelValidation’ parameter to ‘All’ at the time of registration. Later you can also deploy these models for inference in Amazon SageMaker.

### Amazon SageMaker Clarify

[Amazon SageMaker Clarify](https://aws.amazon.com/sagemaker/clarify/) provides purpose-built tools to gain greater insights into your ML models and data, based on metrics such as accuracy, robustness, toxicity, and bias to improve model quality and support responsible AI initiative. With the rise of generative AI, data scientists and ML engineers can leverage publicly available foundation models (FMs) to accelerate speed-to-market. To remove the heavy lifting of evaluating and selecting the right FM for your use case, Amazon SageMaker Clarify supports FM evaluation to help you quickly evaluate, compare, and select the best FM for your use case based on a variety of criteria across different tasks within minutes. It allows you to adopt FMs faster and with confidence.
To perform evaluation we are using the open source library [FMEval](https://github.com/aws/fmeval) that empowers SageMaker Clarify FM model evaluation.

## FM evaluation at scale
This example is build following the best practices explained in the blog post [Operationalize LLM Evaluation at Scale using Amazon SageMaker Clarify and MLOps services](https://aws.amazon.com/blogs/machine-learning/operationalize-llm-evaluation-at-scale-using-amazon-sagemaker-clarify-and-mlops-services/). 




### Install dependencies and import required libraries

In [1]:
#!pip3 install fmeval==0.3.0 --upgrade-strategy only-if-needed --force-reinstall

In [2]:
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.function_step import step
from steps.deploy_llama7b import deploy_llama7b
from steps.finetune_llama7b import finetune_llama7b
from steps.deploy_finetuned_llama7b import deploy_finetuned_llama7b
from steps.selection import selection
from steps.preprocess import preprocess
from steps.evaluation import evaluation
from steps.register import register
from steps.cleanup import cleanup
import os

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


2024-01-19 09:59:24,771	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-01-19 09:59:24,917	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
INFO:datasets:PyTorch version 2.1.2 available.


### Setup source folders and create pipeline preprocess step

In [4]:
os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

pipeline_name = "genai-for-builders-fmops-pipeline"
default_bucket = sagemaker.Session().default_bucket()
main_data_path = f"s3://{default_bucket}/llm-evaluation-at-scale-example"
evaluation_data_path= main_data_path + "/evaluation_dataset_trivia.jsonl"
output_data_path = (main_data_path + "/output_" + pipeline_name)

# You can add your own evaluation dataset code into this step
preprocess_step_ret = step(preprocess, name="preprocess")(evaluation_data_path, output_data_path)

evaluation_results_ret_list = []
model_list = []

sagemaker.config INFO - Fetched defaults config from location: /root/Journey5


### Setup first model: LLama-2-7b from SageMaker Jumpstart

In [5]:
# We setup required model parameters
model_1 = {"model_id": "meta-textgeneration-llama-2-7b",
           "model_version": "*",
           "model_name": "llama-2-7b",
           "endpoint_name": "genai-for-builders-fmops-meta-textgeneration-llama-2-7b",
           "instance_type": "ml.g5.2xlarge",
           "num_instances": 1}

# We save the information of the model in the model_list array for later use
model_list.append(model_1)

### Configure deploy and evaluation pipeline step for the first model

In [6]:
deploy_llama7b_ret = step(deploy_llama7b, name="deploy_llama7b")(model_1["model_id"],
                                                                 model_1["model_version"],
                                                                 model_1["endpoint_name"],
                                                                 model_1["instance_type"],
                                                                 model_1["num_instances"])

# Evaluation step is using the output from preprocess (the S3 location of the evaluation dataset file) 
# and the output of the deploy step (the endpoint name)
evaluate_llama7b_ret = step(evaluation,
                    name="evaluate_llama7b",
                    keep_alive_period_in_seconds=1200
                    )(model_1["model_id"],
                      model_1["model_version"],
                      model_1["model_name"],
                      preprocess_step_ret,
                      deploy_llama7b_ret)

# We save the evaluation output details in the evaluation_results_ret_list array for later use
evaluation_results_ret_list.append(evaluate_llama7b_ret)

### Setup second model: LLama-2-7b from SageMaker Jumpstart, instruction finetuned

In [7]:
# We setup required model parameters
model_2 = {
    "model_id": "meta-textgeneration-llama-2-7b",
    "model_version": "3.0.0",
    "model_name": "llama-2-7b-instruction-tuned",
    "endpoint_name": "genai-for-builders-fmops-meta-instr-llama-2-7b-finetuned",
    "finetune_instance_type": "ml.g5.12xlarge",
    "finetune_num_instances": 1,
    "instance_type": "ml.g5.2xlarge",
    "num_instances": 1,
    "epoch": 1,
    "max_input_length": 512,
    "instruction_tuned": "True",
    "chat_dataset": "False",
    "training_data_path": f"s3://{default_bucket}/llm-evaluation-at-scale-example/train_dataset",
    "is_finetuned_model": True
}

# We save the information of the model in the model_list array for later use
model_list.append(model_2)

### Configure finetune, deploy and evaluation pipeline step for the second model

In [8]:
finetune_ret = step(finetune_llama7b, name="finetune_llama7b_instruction")(model_2["model_id"],
                                                                           model_2["model_version"],
                                                                           model_2["endpoint_name"],
                                                                           model_2["finetune_instance_type"],
                                                                           model_2["finetune_num_instances"],
                                                                           model_2["epoch"],
                                                                           model_2["max_input_length"],
                                                                           model_2["instruction_tuned"],
                                                                           model_2["chat_dataset"],
                                                                           model_2["training_data_path"])

# Deploy step is using the output from the finetune step (the training job name)
deploy_finetuned_llama7b_ret = step(deploy_finetuned_llama7b, 
                                    name="deploy_finetuned_llama7b_instruction")(finetune_ret,
                                                                                 model_2["model_id"],
                                                                                 model_2["endpoint_name"],
                                                                                 model_2["instance_type"],
                                                                                 model_2["num_instances"])

# Evaluation step is using the output from preprocess (the S3 location of the evaluation dataset file) 
# and the output of the deploy step (the endpoint name)
evaluate_finetuned_llama7b_instruction_ret = step(evaluation,
                    name="evaluate_finetuned_llama7b_instr",
                    keep_alive_period_in_seconds=1200,
                    )(model_2["model_id"],
                      model_2["model_version"],
                      model_2["model_name"],
                      preprocess_step_ret,
                      deploy_finetuned_llama7b_ret)

# We save the information of the model in the model_list array for later use
evaluation_results_ret_list.append(evaluate_finetuned_llama7b_instruction_ret)

### Configure selection and model register step

In [9]:
# Evaluation step is using the output from the evaluation steps of all the models
selection_ret = step(selection, name="best_model_selection")(*evaluation_results_ret_list)

model_package_group_name = "GenAIForBuilderFMOpsEvaluationPipeline"
model_package_group_description = "GenAI For Builder FMOps Evaluation Pipeline Model Registry"

# We will register the best model in the model register. The best model name is contained in the return object of the selection step
register_ret = step(register, name="best_model_register")(model_list,
                                                          output_data_path,
                                                          model_package_group_name,
                                                          model_package_group_description,
                                                          selection_ret,
                                                          *evaluation_results_ret_list)

### Cleanup Step

In [10]:
# We need to create a cleanup step for each model. We collect the return objects to add them later in the pipeline creation function
cleanup_ret_list = []

for model in model_list:
    # We append register_ret to connect the register and cleanup steps together
    cleanup_ret = step(cleanup, name="cleanup_"+model["model_name"])(model["endpoint_name"], register_ret)
    cleanup_ret_list.append(cleanup_ret)

### Create requirements.txt
Note: Keeping sagemaker version aligned between the notebook and the container which will run the pipeline steps will help deal with any serialization/deserialization incompatibility issues

In [11]:
%%capture sagemaker_version
!pip show sagemaker | grep Version: | cut -d' ' -f2

In [12]:
if os.path.exists("requirements.txt"):
    os.remove("requirements.txt")

with open('requirements.txt', 'w') as req_file:
    req_file.write("fmeval==0.3.0\n")
    req_file.write("sagemaker=="+str(sagemaker_version)+"\n")
    

### Create and launch the pipeline

In [13]:
from sagemaker import get_execution_role
role = get_execution_role()

pipeline = Pipeline(name=pipeline_name, steps=cleanup_ret_list)
pipeline.upsert(role)
pipeline.start()

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.InstanceType


2024-01-19 09:59:37,402 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-221095015993/genai-for-builders-fmops-pipeline/cleanup_llama-2-7b/2024-01-19-09-59-35-380/function
2024-01-19 09:59:37,489 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-221095015993/genai-for-builders-fmops-pipeline/cleanup_llama-2-7b/2024-01-19-09-59-35-380/arguments
2024-01-19 09:59:37,759 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/tmp/tmpslxhkcgw/requirements.txt'
2024-01-19 09:59:37,786 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-221095015993/genai-for-builders-fmops-pipeline/cleanup_llama-2-7b/2024-01-19-09-59-35-380/pre_exec_script_and_dependencies'
2024-01-19 09:59:37,875 sagemaker.remote_function INFO     Copied user workspace to '/tmp/tmpfbx1ws0_/temp_workspace/sagemaker_remot

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.InstanceType


2024-01-19 09:59:40,153 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-221095015993/genai-for-builders-fmops-pipeline/cleanup_llama-2-7b-instruction-tuned/2024-01-19-09-59-35-380/function
2024-01-19 09:59:40,233 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-221095015993/genai-for-builders-fmops-pipeline/cleanup_llama-2-7b-instruction-tuned/2024-01-19-09-59-35-380/arguments
2024-01-19 09:59:40,316 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/tmp/tmpdexjfbqf/requirements.txt'
2024-01-19 09:59:40,341 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-221095015993/genai-for-builders-fmops-pipeline/cleanup_llama-2-7b-instruction-tuned/2024-01-19-09-59-35-380/pre_exec_script_and_dependencies'


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.InstanceType


...

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.InstanceType


2024-01-19 09:59:54,494 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-221095015993/genai-for-builders-fmops-pipeline/deploy_finetuned_llama7b_instruction/2024-01-19-09-59-35-380/function
2024-01-19 09:59:54,566 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-221095015993/genai-for-builders-fmops-pipeline/deploy_finetuned_llama7b_instruction/2024-01-19-09-59-35-380/arguments
2024-01-19 09:59:54,673 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/tmp/tmprxf2uv0z/requirements.txt'
2024-01-19 09:59:54,703 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-221095015993/genai-for-builders-fmops-pipeline/deploy_finetuned_llama7b_instruction/2024-01-19-09-59-35-380/pre_exec_script_and_dependencies'


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.InstanceType


2024-01-19 09:59:56,551 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-221095015993/genai-for-builders-fmops-pipeline/finetune_llama7b_instruction/2024-01-19-09-59-35-380/function
2024-01-19 09:59:56,649 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-221095015993/genai-for-builders-fmops-pipeline/finetune_llama7b_instruction/2024-01-19-09-59-35-380/arguments
2024-01-19 09:59:56,756 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/tmp/tmph1e8g8cu/requirements.txt'
2024-01-19 09:59:56,785 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-221095015993/genai-for-builders-fmops-pipeline/finetune_llama7b_instruction/2024-01-19-09-59-35-380/pre_exec_script_and_dependencies'
2024-01-19 09:59:57,230 sagemaker.remote_function INFO     Uploading serialized function code to s3://

_PipelineExecution(arn='arn:aws:sagemaker:us-east-1:221095015993:pipeline/genai-for-builders-fmops-pipeline/execution/o6vyqonudi0i', sagemaker_session=<sagemaker.session.Session object at 0x7ff6523c8460>)